In [ ]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.4 MB/s eta 0:00:00


In [ ]:
import os
import subprocess
import re
from yt_dlp import YoutubeDL
from google import genai


DOWNLOAD_FOLDER = "downloads"
os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

def sanitize_filename(filename):
    # Hapus karakter yang tidak valid untuk nama file
    return re.sub(r'[\\/*?:"<>|]', "", filename)

def clean_transcript(raw_text):
    raw_text = re.sub(r'^WEBVTT.*?\n', '', raw_text, flags=re.DOTALL)
    raw_text = re.sub(r'<\d{2}:\d{2}:\d{2}\.\d{3}>', '', raw_text)

    lines = raw_text.splitlines()
    cleaned_lines = []
    for line in lines:
        if re.match(r"^\d\d:\d\d:\d\d\.\d{3} -->", line):
            continue
        line = re.sub(r"</?c>", "", line)
        line = line.strip()
        if line:
            cleaned_lines.append(line)
    return " ".join(cleaned_lines)

def gemini_summarize_transcipt(raw_text):

  client = genai.Client(api_key="your_api_key")

  response = client.models.generate_content(
      model="gemini-2.0-flash", contents=f"summarize using first principle thinking this text and give point emoji ({raw_text})"
  )
  return response.text

def openai_summarize_transcipt(raw_text):
  return

def divider_text(raw_text):
  # make optimal LLM Response
  return

def get_ydl_opts():
    """Return YouTube downloader options configuration"""
    return {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(DOWNLOAD_FOLDER, '%(id)s.%(ext)s'),
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'writesubtitles': True,
        'writeautomaticsub': True,
        'subtitleslangs': ['en'],
    }

def download_video(url):
    """Download video and return info dictionary"""
    with YoutubeDL(get_ydl_opts()) as ydl:
        return ydl.extract_info(url, download=True)

def get_file_paths(info):
    """Generate file paths based on video info"""
    video_title = sanitize_filename(info.get('title', info['id']))
    return {
        'audio_file': os.path.join(DOWNLOAD_FOLDER, f"{video_title}.mp3"),
        'subtitle_file': os.path.join(DOWNLOAD_FOLDER, f"{video_title}.en.vtt"),
        'transcript_txt': os.path.join(DOWNLOAD_FOLDER, f"{video_title}.txt"),
        'downloaded_mp3': os.path.join(DOWNLOAD_FOLDER, f"{info['id']}.mp3"),
        'downloaded_sub': os.path.join(DOWNLOAD_FOLDER, f"{info['id']}.en.vtt"),
    }

def rename_downloaded_files(info, file_paths):
    """Rename downloaded files from ID-based to title-based names"""
    if os.path.exists(file_paths['downloaded_mp3']):
        os.rename(file_paths['downloaded_mp3'], file_paths['audio_file'])

    if os.path.exists(file_paths['downloaded_sub']):
        os.rename(file_paths['downloaded_sub'], file_paths['subtitle_file'])

def get_transcript_from_subtitle(subtitle_file):
    """Read and clean transcript from subtitle file"""
    with open(subtitle_file, "r", encoding="utf-8") as f:
        raw_sub = f.read()
    return clean_transcript(raw_sub)

def get_transcript_from_whisper(audio_file, output_dir):
    """Generate transcript using Whisper"""
    cmd = f"whisper \"{audio_file}\" --language en --task transcribe --output_format txt --output_dir \"{output_dir}\""
    subprocess.run(cmd, shell=True, check=True)

def read_transcript_file(transcript_file):
    """Read transcript from text file"""
    if os.path.exists(transcript_file):
        with open(transcript_file, "r", encoding="utf-8") as f:
            return f.read()
    return ""
def save_transcript_to_text():
  return

def save_summarize_to_txt():
  return
def download_and_transcribe(url):
    """Main function to download and transcribe video"""
    try:
        # Step 1: Download video
        info = download_video(url)

        # Step 2: Prepare file paths
        file_paths = get_file_paths(info)

        # Step 3: Rename downloaded files
        rename_downloaded_files(info, file_paths)

        # Step 4: Get transcript (from subtitles or Whisper)
        if os.path.exists(file_paths['subtitle_file']):
            transcript = get_transcript_from_subtitle(file_paths['subtitle_file'])
        else:
            get_transcript_from_whisper(file_paths['audio_file'], DOWNLOAD_FOLDER)
            transcript = read_transcript_file(file_paths['transcript_txt'])
            if not transcript:
                print("Whisper transcript file tidak ditemukan.")

        # Step 5: Summarize and return
        return gemini_summarize_transcipt(transcript), None

    except Exception as e:
        return "", str(e)

In [ ]:
download_and_transcribe("https://youtu.be/xfOT2elC2Ok?si=0tqj1IXmdDMZKy6I")

[youtube] Extracting URL: https://youtu.be/xfOT2elC2Ok?si=0tqj1IXmdDMZKy6I
[youtube] xfOT2elC2Ok: Downloading webpage
[youtube] xfOT2elC2Ok: Downloading tv client config
[youtube] xfOT2elC2Ok: Downloading tv player API JSON
[youtube] xfOT2elC2Ok: Downloading ios player API JSON
[youtube] xfOT2elC2Ok: Downloading m3u8 information
[info] xfOT2elC2Ok: Downloading subtitles: en
[info] xfOT2elC2Ok: Downloading 1 format(s): 251-4
[info] Writing video subtitles to: downloads/xfOT2elC2Ok.en.vtt
[download] Destination: downloads/xfOT2elC2Ok.en.vtt
[download] 100% of  175.80KiB in 00:00:00 at 3.28MiB/s
[download] Destination: downloads/xfOT2elC2Ok.webm
[download] 100% of   17.04MiB in 00:00:00 at 45.41MiB/s  
[ExtractAudio] Destination: downloads/xfOT2elC2Ok.mp3
Deleting original file downloads/xfOT2elC2Ok.webm (pass -k to keep)


("The core idea being tested is whether it's possible to create a profitable online store using only free or very cheap AI tools and a limited budget.\n\n*   💰 **Hypothesis:** People claim to make thousands in passive income via AI-powered dropshipping stores, but is it real?\n*   ⚙️ **Experiment:** Build a Shopify store using free AI tools, keeping setup costs under $1, and operate solely from a laptop within an hour.\n*   🧩 **Tools:** Free AI store builder, Shopify (free trial initially), AutoDS (99 cent trial).\n*   🛍️ **Product Selection:** AI-suggested fitness products sourced via AutoDS.\n*   📢 **Marketing:** Micro-influencer collaboration for promotion.\n*   📈 **Results:** 10 genuine sales after influencer marketing leading to profit after expenses.\n*   ✅ **Conclusion:** While not life-changing, the experiment shows potential for scaling and passive income generation using AI and dropshipping with minimal initial investment.\n",
 None)

In [ ]:
!pip install -q -U google-genai

In [ ]:
from google import genai

client = genai.Client(api_key="your_api_key")

response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Halo siapa namamu ?"
)
print(response.text)

Sebagai model bahasa besar, saya dikembangkan di Google. Saya tidak punya nama pribadi. Anda bisa memanggil saya Google AI.



In [ ]:
import os
import subprocess
import re
from typing import Dict, Tuple, Optional, List
from yt_dlp import YoutubeDL
from pathlib import Path

try:
    import google.genai as genai
except ImportError:
    genai = None

# Versi yt_dlp
import os
import subprocess
import re
from typing import Dict, Tuple, Optional, List
from yt_dlp import YoutubeDL
from pathlib import Path

try:
    import google.genai as genai
except ImportError:
    genai = None

# Versi yt_dlp
print("yt_dlp version:", YoutubeDL.__version__)

# Versi Google Generative AI (genai)
if genai:
    try:
        print("google.genai version:", genai.__version__)
    except AttributeError:
        print("google.genai version: Tidak tersedia via __version__")
else:
    print("google.genai tidak tersedia")

# Versi subprocess (builtin, tidak punya version langsung)
print("subprocess: built-in module")

# Cek versi OS-level tools (ffmpeg)
def check_tool_version(tool: str):
    try:
        result = subprocess.run([tool, "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        return result.stdout.split("\n")[0]
    except Exception as e:
        return f"Error checking {tool}: {str(e)}"

print("ffmpeg version:", check_tool_version("ffmpeg"))

# Versi Google Generative AI (genai)
if genai:
    try:
        print("google.genai version:", genai.__version__)
    except AttributeError:
        print("google.genai version: Tidak tersedia via __version__")
else:
    print("google.genai tidak tersedia")

# Versi subprocess (builtin, tidak punya version langsung)
print("subprocess: built-in module")

# Cek versi OS-level tools (ffmpeg)
def check_tool_version(tool: str):
    try:
        result = subprocess.run([tool, "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        return result.stdout.split("\n")[0]
    except Exception as e:
        return f"Error checking {tool}: {str(e)}"

print("ffmpeg version:", check_tool_version("ffmpeg"))



AttributeError: type object 'YoutubeDL' has no attribute '__version__'

In [ ]:
import os
import subprocess
import re
from typing import Dict, Tuple, Optional, List
from yt_dlp import YoutubeDL
from google import genai
from pathlib import Path

# Constants
DOWNLOAD_FOLDER = "downloads"
GEMINI_MODEL = "gemini-2.0-flash"  # Model terbaru yang mendukung context length lebih panjang
MAX_DIRECT_SUMMARY_LENGTH = 30000  # Batas untuk summarisasi langsung
CHUNK_SIZE = 15000  # Ukuran chunk untuk teks sangat panjang
MIN_SUMMARY_RATIO = 0.3  # Rasio kompresi minimum untuk summary

class VideoProcessor:
    def __init__(self):
        os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)
        self.gemini_client = None
    def initialize_gemini(self, api_key: str):
        """Initialize the Gemini client with API key"""
        try:
            # Initialize the Gemini client
            self.gemini_client = genai.Client(api_key=api_key)

            # Test the connection with a simple prompt
            test_prompt = "Hello, Gemini! Please respond with 'OK' if you're working."
            response = self.gemini_client.models.generate_content(
              model="gemini-2.0-flash", contents= test_prompt
            )

            # Verify we got a response
            if not response.text:
                raise RuntimeError("Gemini API connection failed - no response received")

            print("Gemini API initialized successfully")
        except Exception as e:
            raise RuntimeError(f"Failed to initialize Gemini client: {str(e)}")

    @staticmethod
    def sanitize_filename(filename: str) -> str:
        """Remove invalid characters from filename"""
        return re.sub(r'[\\/*?:"<>|]', "", filename)

    @staticmethod
    def clean_transcript(raw_text: str) -> str:
        """Clean and format the raw transcript text"""
        # Remove WEBVTT header and timestamps
        raw_text = re.sub(r'^WEBVTT.*?\n', '', raw_text, flags=re.DOTALL)
        raw_text = re.sub(r'<\d{2}:\d{2}:\d{2}\.\d{3}>', '', raw_text)

        lines = raw_text.splitlines()
        cleaned_lines = []

        for line in lines:
            if re.match(r"^\d\d:\d\d:\d\d\.\d{3} -->", line):
                continue
            line = re.sub(r"</?c>", "", line).strip()
            if line:
                cleaned_lines.append(line)

        return " ".join(cleaned_lines)

    def smart_summarize(self, text: str) -> str:
        """
        Hybrid summarization approach dengan strategi:
        1. Untuk teks pendek (<30k chars): summarisasi langsung
        2. Untuk teks medium (30k-100k): chunk processing
        3. Untuk teks sangat panjang (>100k): keypoint extraction + summarization
        """
        text_length = len(text)

        if text_length <= MAX_DIRECT_SUMMARY_LENGTH:
            return self._direct_summarize(text)
        elif text_length <= 100000:
            return self._chunked_summarize(text)
        else:
            return self._keypoint_based_summarize(text)

    def _direct_summarize(self, text: str) -> str:
        """Summarize text directly (for short texts)"""
        prompt = (
            "Buat ringkasan mendalam dari teks berikut menggunakan prinsip first-principles.\n"
            "Gunakan struktur:\n"
            "1. Poin-poin kunci (emoji)\n"
            "2. Insight utama\n"
            "3. Rekomendasi/aplikasi praktis\n\n"
            f"Teks:\n{text}"
        )

        response = self.gemini_client.models.generate_content(
            model=GEMINI_MODEL,
            contents=prompt
        )
        return response.text

    def _chunked_summarize(self, text: str) -> str:
        """Process text in chunks for medium-length texts"""
        chunks = self._split_text_into_chunks(text, CHUNK_SIZE)
        partial_summaries = []

        for i, chunk in enumerate(chunks):
            prompt = (
                f"Buat ringkasan parsial untuk bagian {i+1}/{len(chunks)}:\n"
                f"{chunk}\n\n"
                "Fokus pada poin-poin baru yang belum tercakup di summary sebelumnya."
            )

            response = self.gemini_client.models.generate_content(
                model=GEMINI_MODEL,
                contents=prompt
            )
            partial_summaries.append(response.text)

        # Gabungkan dan refine summary akhir
        combined = "\n\n".join(partial_summaries)
        refined_prompt = (
            "Gabungkan ringkasan-ringkasan parsial berikut menjadi satu dokumen koheren:\n"
            f"{combined}\n\n"
            "Pertahankan struktur:\n"
            "1. Poin kunci\n"
            "2. Insight\n"
            "3. Aplikasi praktis"
        )

        final_response = self.gemini_client.models.generate_content(
            model=GEMINI_MODEL,
            contents=refined_prompt
        )
        return final_response.text

    def _keypoint_based_summarize(self, text: str) -> str:
        """Keypoint extraction + summarization for very long texts"""
        # Langkah 1: Ekstrak poin kunci dari berbagai bagian teks
        sample_points = [
            text[:20000],  # Awal
            text[len(text)//3 : len(text)//3 + 20000],  # Tengah
            text[-20000:],  # Akhir
            text[::len(text)//5][:20000]  # Sampel acak
        ]

        keypoints = []
        for sample in sample_points:
            prompt = (
                "Identifikasi 5-7 poin kunci paling penting dari teks berikut:\n"
                f"{sample}\n"
                "Format: - [Poin] (alasan pentingnya)"
            )
            response = self.gemini_client.models.generate_content(
                model=GEMINI_MODEL,
                contents=prompt
            )
            keypoints.append(response.text)

        # Langkah 2: Gabungkan dan buat summary akhir
        combined_keypoints = "\n\n".join(keypoints)
        summary_prompt = (
            "Buat ringkasan eksekutif berdasarkan poin-poin kunci berikut:\n"
            f"{combined_keypoints}\n\n"
            "Struktur:\n"
            "1. Overview\n"
            "2. Poin Strategis (emoji)\n"
            "3. Rekomendasi\n"
            "4. Potensi Implementasi"
        )

        final_response = self.gemini_client.models.generate_content(
            model=GEMINI_MODEL,
            contents=summary_prompt
        )
        return final_response.text

    def _split_text_into_chunks(self, text: str, chunk_size: int) -> List[str]:
        """Split text into chunks with overlap to maintain context"""
        chunks = []
        start = 0
        while start < len(text):
            end = start + chunk_size
            # Tambahkan overlap 10% untuk mempertahankan konteks
            overlap = int(chunk_size * 0.1)
            if end < len(text):
                # Cari titik pemisah alami (akhir paragraf)
                split_pos = text.rfind('\n', start + chunk_size - overlap, start + chunk_size + overlap)
                end = split_pos if split_pos != -1 else end
            chunks.append(text[start:end])
            start = end
        return chunks

    def save_text_to_file(self, text: str, file_path: str) -> None:
        """
        Save text content to a file with proper error handling and atomic write
        """
        try:
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(file_path), exist_ok=True)

            # Write to temporary file first to ensure atomic operation
            temp_path = file_path + '.tmp'
            with open(temp_path, 'w', encoding='utf-8') as f:
                f.write(text)

            # Rename temporary file to target file
            os.replace(temp_path, file_path)
        except IOError as e:
            raise RuntimeError(f"Gagal menyimpan file {file_path}: {str(e)}")
        except Exception as e:
            raise RuntimeError(f"Error tak terduga saat menyimpan file: {str(e)}")
    def download_video(self, url: str) -> Dict:
        """Download video and return info dictionary"""
        try:
            with YoutubeDL(self.get_ydl_opts()) as ydl:
                return ydl.extract_info(url, download=True)
        except Exception as e:
            raise RuntimeError(f"Failed to download video: {str(e)}")

    def get_ydl_opts(self) -> Dict:
        """Return YouTube downloader options configuration"""
        return {
            'format': 'bestaudio/best',
            'outtmpl': os.path.join(DOWNLOAD_FOLDER, '%(id)s.%(ext)s'),
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'writesubtitles': True,
            'writeautomaticsub': True,
            'subtitleslangs': ['en'],
            'quiet': True,
            'no_warnings': True,
        }
    def get_file_paths(self, info: Dict) -> Dict:
        """Generate file paths based on video info"""
        video_title = self.sanitize_filename(info.get('title', info['id']))
        base_path = os.path.join(DOWNLOAD_FOLDER, video_title)

        return {
            'audio_file': f"{base_path}.mp3",
            'subtitle_file': f"{base_path}.en.vtt",
            'transcript_txt': f"{base_path}.txt",
            'summary_txt': f"{base_path}_summary.txt",
            'downloaded_mp3': os.path.join(DOWNLOAD_FOLDER, f"{info['id']}.mp3"),
            'downloaded_sub': os.path.join(DOWNLOAD_FOLDER, f"{info['id']}.en.vtt"),
        }

    def rename_downloaded_files(self, info: Dict, file_paths: Dict) -> None:
        """Rename downloaded files from ID-based to title-based names"""
        try:
            if os.path.exists(file_paths['downloaded_mp3']):
                os.rename(file_paths['downloaded_mp3'], file_paths['audio_file'])

            if os.path.exists(file_paths['downloaded_sub']):
                os.rename(file_paths['downloaded_sub'], file_paths['subtitle_file'])
        except Exception as e:
            raise RuntimeError(f"Failed to rename files: {str(e)}")

    def get_transcript_from_subtitle(self, subtitle_file: str) -> str:
        """Read and clean transcript from subtitle file"""
        try:
            with open(subtitle_file, "r", encoding="utf-8") as f:
                raw_sub = f.read()
            return self.clean_transcript(raw_sub)
        except Exception as e:
            raise RuntimeError(f"Failed to process subtitle file: {str(e)}")

    def get_transcript_from_whisper(self, audio_file: str, output_dir: str) -> str:
        """Generate transcript using Whisper"""
        try:
            cmd = [
                "whisper",
                audio_file,
                "--language", "en",
                "--task", "transcribe",
                "--output_format", "txt",
                "--output_dir", output_dir
            ]
            subprocess.run(cmd, check=True, capture_output=True)

            # Find the generated transcript file
            base_name = Path(audio_file).stem
            transcript_file = os.path.join(output_dir, f"{base_name}.txt")

            if os.path.exists(transcript_file):
                with open(transcript_file, "r", encoding="utf-8") as f:
                    return f.read()
            return ""
        except subprocess.CalledProcessError as e:
            raise RuntimeError(f"Whisper failed: {e.stderr.decode('utf-8')}")
        except Exception as e:
            raise RuntimeError(f"Failed to run Whisper: {str(e)}")

    def process_video(self, url: str, gemini_api_key: str) -> Dict[str, str]:
        """Main processing pipeline with enhanced error handling"""
        result = {
            'status': 'error',
            'transcript': '',
            'summary': '',
            'audio_path': '',
            'error': ''
        }

        try:
            # Initialize Gemini
            self.initialize_gemini(gemini_api_key)

            # Download video
            info = self.download_video(url)
            file_paths = self.get_file_paths(info)
            self.rename_downloaded_files(info, file_paths)

            # Get transcript
            if os.path.exists(file_paths['subtitle_file']):
                transcript = self.get_transcript_from_subtitle(file_paths['subtitle_file'])
            else:
                transcript = self.get_transcript_from_whisper(file_paths['audio_file'], DOWNLOAD_FOLDER)

            # Save transcript
            self.save_text_to_file(transcript, file_paths['transcript_txt'])

            # Generate summary
            summary = self.smart_summarize(transcript)
            self.save_text_to_file(summary, file_paths['summary_txt'])

            # Update result
            result.update({
                'status': 'success',
                'transcript': transcript,
                'summary': summary,
                'audio_path': file_paths['audio_file']
            })

        except Exception as e:
            result['error'] = str(e)
            # Log the error
            error_log = f"Error processing {url}: {str(e)}\n\n"
            self.save_text_to_file(error_log, os.path.join(DOWNLOAD_FOLDER, 'errors.log'))

        return result


In [ ]:
processor = VideoProcessor()
result = processor.process_video(
    url="https://youtu.be/xfOT2elC2Ok?si=0tqj1IXmdDMZKy6I",
    gemini_api_key="your_api_key"
)

if result['status'] == 'success':
    print("Ringkasan:", result['summary'])
    print("Full transcript tersedia di:", result['audio_path'].replace('.mp3', '.txt'))
else:
    print("Error:", result['error'])

Gemini API initialized successfully
Ringkasan: Berikut adalah gabungan ringkasan parsial menjadi dokumen yang koheren, dengan struktur yang mempertimbangkan poin kunci, insight, dan aplikasi praktis:

**Uji Coba Penghasilan Pasif: Membuat Toko Dropshipping Shopify dengan AI Gratis**

**Poin Kunci:**

*   Tujuan: Menguji klaim bahwa orang dapat menghasilkan ribuan dolar per bulan melalui penghasilan pasif dengan dropshipping dari toko Shopify yang dibuat dengan AI.
*   Metode: Membuat toko Shopify menggunakan alat AI gratis, dengan batasan biaya kurang dari $1, dikerjakan dari laptop, dan diselesaikan dalam waktu kurang dari satu jam.
*   Alat yang Digunakan: "Build Your Store" (aplikasi AI untuk membangun toko Shopify), AutoDS (platform untuk mencari dan mengelola produk dropshipping), dan Shopify (platform e-commerce).
*   Niche: Olahraga & Kebugaran, dipilih karena permintaan tinggi, terutama di kalangan anak muda.
*   Promosi: Kolaborasi dengan micro-influencer di Instagram.

**Pros

In [ ]:
processor = VideoProcessor()
result = processor.process_video(
    url="https://www.youtube.com/watch?v=EaZvEA1ZcwI&t=457s",
    gemini_api_key="your_api_key"
)

if result['status'] == 'success':
    print("Ringkasan:", result['summary'])
    print("Full transcript tersedia di:", result['audio_path'].replace('.mp3', '.txt'))
else:
    print("Error:", result['error'])

Gemini API initialized successfully
Ringkasan: Absolutely! Here's a breakdown of the video content using first principles, structured as requested:

**1. Poin-poin Kunci 🔑**

*   **CV 📝:** Desain menarik, foto diri, data lengkap (IPK penting utk *fresh graduate*), sertifikasi (khususnya bidang IT).
*   **Psikotes & IQ Test 🧠:** Jangan asal jawab. Konsistensi jawaban penting. Jangan berbohong atau membuat jawaban yang tidak sesuai dengan kepribadian.
*   **HR Interview 🗣️:** Tepat waktu, penampilan rapi. Jawab pertanyaan dengan jujur tentang tujuan hidup dan pencapaian. HR melihat kepribadian.
*   **User Interview 🧑‍💻:** Persiapkan *portfolio* dan materi pendukung lainnya. Jawab pertanyaan teknis, kemampuan *coding* dan logika dites.
*   **C-Level Interview (CEO) 💼:** Jangan gugup. Jujur tentang diri sendiri dan pengalaman. CEO ingin melihat kecocokan kepribadian dan *chemistry*.
*   **Fokus Utama 🎯:** Kembangkan *skill* dan kepribadian (*attitude*). Jalur aplikasi (job connector, dll.)

In [ ]:
processor = VideoProcessor()
result = processor.process_video(
    url="https://www.youtube.com/watch?v=kNPEGMhlRUg&t=40s",
    gemini_api_key="your_api_key"
)

if result['status'] == 'success':
    print("Ringkasan:", result['summary'])
    print("Full transcript tersedia di:", result['audio_path'].replace('.mp3', '.txt'))
else:
    print("Error:", result['error'])

Gemini API initialized successfully
Ringkasan: ## Produktivitas Elon Musk: Manajemen Waktu, Proses, dan Pengambilan Keputusan

Video ini membahas produktivitas Elon Musk dan membagi rahasianya menjadi tiga kategori utama: manajemen waktu, manajemen proses, dan pengambilan keputusan. Musk menggabungkan kerja keras dengan kerja cerdas, berbeda dengan mentalitas "Hassle Pride Complex" yang mengagungkan lembur tanpa efektivitas. Dengan memahami dan menerapkan prinsip-prinsip ini, bisnis dari berbagai skala dapat meningkatkan efisiensi dan mencapai pertumbuhan.

**1. Manajemen Waktu:**

*   **Poin Kunci:**
    *   **Time Blocking Ekstrim:** Membagi hari menjadi blok 5 menit untuk aktivitas spesifik.
    *   **Jadwal Harian Terstruktur: Metode Satu Hari Satu Perusahaan:** Mengkhususkan hari untuk perusahaan tertentu untuk menghindari *context switching*.
    *   **Rutinitas Harian yang Efisien:** Membangun rutinitas untuk mengatur produktivitas.
    *   **Prioritas untuk Tidur:** Mengutamaka